## Тестирование класса FMU_model

In [1]:
import time

In [2]:
import sys
sys.path.append('../')

In [3]:
from models_server import db
from models_server.models import Model, Variable, Measurement, Achive, Task

In [4]:
from models_server.fmu_model import *

In [5]:
import pandas as pd
import numpy as np

In [6]:
def write_to_db(df, db, diff, time):
    df['to_db'] = True
    df.loc[abs((df.sol - df.prev_sol) / df.sol) > diff, 'to_db'] = False
    df.loc[(df.sol - df.prev_sol) == 0, 'to_db'] = False
    flag = 0
    for index, row in df.loc[df.to_db, ('id', 'sol')].iterrows():
        db.session.add(Measurement(achive_id=achive.id, variable_id=row['id'], value=row['sol'], time=time))
        flag += 1
    db.session.commit()
    print('На шаге {} записано {} переменных'.format(time, flag))

In [7]:
user_id = 3
model_id = 1

In [8]:
# Создаем задачу в базе данных
task = Task(model_id=model_id, user_id=user_id)
db.session.add(task)
db.session.commit()

In [9]:
# Инстанс классса
new_model = fmu_model('../fmu/TPPSim02.Pipes.Tests.PipeChannelFMU_Test.fmu')

In [10]:
# Инициализация модели
# Здесь должно быть задание входных параметров
new_model.initialize()

Инициализация модели выполнена


In [11]:
# Создание архива в базе данных
achive = Achive(user_id=user_id, task_id=task.id, model_id=model_id)
db.session.add(achive)
db.session.commit()

In [12]:
# Получаем значения переменных
db_model = Model.query.get(1)
var = db_model.variables.all()

In [13]:
df = pd.DataFrame({
    'id':[i.id for i in var],
    'name':[i.name for i in var],
    'vref':[new_model.model.get_variable_valueref(i.name) for i in var],
    'sol':np.nan,
    'prev_sol':np.nan
})

In [14]:
df['sol'] = new_model.model.get_real(df.vref)

In [ ]:
#     time.sleep(1)
while True:
    # Запись переменных в БД
    write_to_db(df, db, 1e-4, new_model.model.time)
    # Итегрирование
    new_model.integrate()
    # Сохранение предыдущего и текущего решений
    df['prev_sol'] = df.sol
    df.sol = new_model.model.get_real(df.vref)

На шаге 0 записано 473 переменных
Точка -1
Точка 0
Точка 1
Точка 2
Точка 3
На шаге 0.01 записано 60 переменных
Точка -1
Точка 0
Точка 1
Точка 2
Точка 3
На шаге 0.02 записано 90 переменных
Точка -1
Точка 0
Точка 1
Точка 2
Точка 3
На шаге 0.03 записано 94 переменных
Точка -1
Точка 0
Точка 1
Точка 2
Точка 3
На шаге 0.04 записано 109 переменных
Точка -1
Точка 0
Точка 1
Точка 2
Точка 3
На шаге 0.05 записано 74 переменных
Точка -1
Точка 0
Точка 1
Точка 2
Точка 3
На шаге 0.060000000000000005 записано 82 переменных
Точка -1
Точка 0
Точка 1
Точка 2
Точка 3
На шаге 0.07 записано 74 переменных
Точка -1
Точка 0


In [ ]:
new_model.model.get_log()

In [ ]:
# for m in Measurement.query.all():
#     db.session.delete(m)
# db.session.commit()